# DCOPF Verification

Prepared by [Jinning Wang](https://jinningwang.github.io).

## Conclusion

For test cases, DCOPF results from AMS are identical to that from PYPOWER.

NOTE: This verification works with PYPOWER ***v5.1.16***, where ***v5.1.17*** yields unexpected results.

In [1]:
import datetime

import numpy as np
import pandas as pd

import ams
import andes

import pypower.api as pyp

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams: {ams.__version__}')

Last run time: 2024-11-24 21:03:09
ams: 0.9.12


In [3]:
ams.config_logger(stream_level=30)

## Generator Output

Using built-in MATPOWER cases as inputs.

In [4]:
cases = [
    ams.get_case('matpower/case14.m'),
    ams.get_case('matpower/case39.m'),
    ams.get_case('matpower/case118.m'),
    ams.get_case('npcc/npcc.m'),
    ams.get_case('wecc/wecc.m'),
    ams.get_case('matpower/case300.m'),]

case_names = [case.split('/')[-1].split('.')[0] for case in cases]

In [5]:
ams_obj = np.zeros(len(cases))
pyp_obj = np.zeros(len(cases))

for i, case in enumerate(cases):
    sp = ams.load(case, setup=True)
    sp.DCOPF.init()
    sp.DCOPF.run(solver='CLARABEL')
    ams_obj[i] = sp.DCOPF.obj.v

    ppc = ams.io.pypower.system2ppc(sp)
    ppopt = pyp.ppoption(VERBOSE=0, OUT_ALL=0, PF_ALG=1, OPF_ALG_DC=200)
    ppc_sol = pyp.rundcopf(ppc, ppopt)
    pyp_obj[i] = ppc_sol['f']

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> solved as optimal in 0.0065 seconds, converged in 11 iterations with CLARABEL.
Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> solved as optimal in 0.0077 seconds, converged in 8 iterations with CLARABEL.
Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> solved as optimal in 0.0144 seconds, converged in 11 iterations with CLARABEL.
Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> solved as optimal in 0.0164 seconds, converged in 12 iterations with CLARABEL.
Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> solved as optimal in 0.0160 seconds, converged in 12 iterations with CLARABEL.
Buildi

In [6]:
np.allclose(ams_obj, pyp_obj, atol=1e-6)

True

## LMP

In [7]:
sp2 = ams.load(ams.get_case('pglib/pglib_opf_case39_epri__api.m'),
               setup=True,
               no_output=True,
               default_config=True)

In [8]:
sp2.DCOPF.run(solver='CLARABEL')

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> solved as optimal in 0.0072 seconds, converged in 13 iterations with CLARABEL.


True

In [9]:
ppc2 = ams.io.pypower.system2ppc(sp2)
ppc2_sol = pyp.rundcopf(ppc2, ppopt)

Nodal price

In [10]:
np.allclose(sp2.DCOPF.pi.v / sp2.config.mva,
            ppc2_sol['bus'][:, 13],
            atol=1e-6)

True

Bus angle

In [11]:
np.allclose(sp2.DCOPF.aBus.v * andes.shared.rad2deg,
            ppc2_sol['bus'][:, 8],
            atol=1e-6)

True

In [12]:
pd.DataFrame({'ams_LMP': sp2.DCOPF.pi.v / sp2.config.mva,
              'pyp_LMP': ppc2_sol['bus'][:, 13],
              'ams_aBus': sp2.DCOPF.aBus.v * andes.shared.rad2deg,
              'pyp_aBus': ppc2_sol['bus'][:, 8]}).round(4)

,ams_LMP,pyp_LMP,ams_aBus,pyp_aBus
0,29.9988,29.9988,-25.6184,-25.6184
1,34.2147,34.2147,-26.8519,-26.8519
2,35.6012,35.6012,-30.4965,-30.4965
3,38.1542,38.1542,-28.8548,-28.8548
4,20.1311,20.1311,-24.4545,-24.4545
5,20.9823,20.9823,-22.9110,-22.9110
6,21.0918,21.0918,-26.4821,-26.4821
7,21.1466,21.1466,-27.2531,-27.2531
8,24.8701,24.8701,-24.5329,-24.5329
9,24.8047,24.8047,-21.2851,-21.2851
